In [1]:
import numpy as np
import pandas as pd

# Parameters
num_samples = 10
num_features = 8

# Generate random data
X = np.random.rand(num_samples, num_features)  # Features
y = np.random.choice([0, 1], size=(num_samples,))  # Binary labels (0 or 1)

# Create DataFrame
feature_columns = [f'feature_{i+1}' for i in range(num_features)]
df = pd.DataFrame(X, columns=feature_columns)
df['label'] = y

# Export to CSV
output_filename = r'C:\Users\muthu\Downloads\sample.csv'
df.to_csv(output_filename, index=False)

print(f"Dataset exported to {output_filename}")

Dataset exported to C:\Users\muthu\Downloads\sample.csv


In [2]:
df

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,label
0,0.889916,0.299667,0.937930,0.248298,0.146758,0.500100,0.381092,0.980770,0
1,0.504135,0.801872,0.540422,0.270302,0.109999,0.097354,0.498183,0.209367,0
2,0.358965,0.904812,0.133417,0.185259,0.153765,0.441377,0.311721,0.247051,1
3,0.409503,0.069783,0.850485,0.913073,0.900766,0.394887,0.135831,0.216830,0
4,0.683421,0.277698,0.012079,0.841432,0.238404,0.972608,0.836050,0.435805,0
5,0.051418,0.859557,0.214035,0.036540,0.884123,0.298437,0.525779,0.845549,0
6,0.755669,0.026058,0.153110,0.593689,0.465011,0.899505,0.637755,0.496283,1
7,0.723328,0.133783,0.433384,0.598317,0.018134,0.989682,0.398346,0.415757,1
8,0.549673,0.299779,0.782107,0.551414,0.038046,0.457786,0.384723,0.825939,1
9,0.001736,0.401760,0.752279,0.036619,0.674771,0.143152,0.926781,0.052970,1


In [5]:
IBMQ_TOKEN="66755fbc0167ef227e9fa10a64a03b66a8bbb3500cc1dc0d3adcc9680e3142d486f8a4f7d537f068dc4e26a46b3c3a69ce0976f8ffbb9f5edecdeaddb70d8844"

In [7]:
import pennylane as qml
from qiskit_aer import AerSimulator

qubits = 8
dev_aer = qml.device("qiskit.aer", wires=qubits)
dev_basicsim = qml.device("qiskit.basicsim", wires=qubits)
try:
    dev_remote = qml.device("qiskit.remote", wires=qubits, backend=AerSimulator())
except Exception as e:
    print(e)

ImportError: cannot import name 'convert_to_target' from 'qiskit.providers' (C:\Users\muthu\anaconda3\Lib\site-packages\qiskit\providers\__init__.py)

In [99]:
dev_aer = qml.device("qiskit.aer", wires=qubits)
from qiskit_aer import Aer

print(Aer.backends())

[AerSimulator('aer_simulator'), AerSimulator('aer_simulator_statevector'), AerSimulator('aer_simulator_density_matrix'), AerSimulator('aer_simulator_stabilizer'), AerSimulator('aer_simulator_matrix_product_state'), AerSimulator('aer_simulator_extended_stabilizer'), AerSimulator('aer_simulator_unitary'), AerSimulator('aer_simulator_superop'), QasmSimulator('qasm_simulator'), StatevectorSimulator('statevector_simulator'), UnitarySimulator('unitary_simulator')]


In [101]:
from qiskit_ibm_runtime import QiskitRuntimeService
service = QiskitRuntimeService(channel='ibm_quantum', token=IBMQ_TOKEN)
# List available backends
backends = service.backends()
print(backends)

[<IBMBackend('ibm_brisbane')>, <IBMBackend('ibm_kyiv')>, <IBMBackend('ibm_sherbrooke')>]


In [102]:
backend = service.backend('ibm_brisbane')
status = backend.status()
print(f"Backend Name: {status.backend_name}")
print(f"Backend Version: {status.backend_version}")
print(f"Operational: {status.operational}")
print(f"Pending Jobs: {status.pending_jobs}")
print(f"Status Message: {status.status_msg}")

Backend Name: ibm_brisbane
Backend Version: 
Operational: True
Pending Jobs: 21
Status Message: active


In [105]:
try:
    # Our device supports a maximum of 31 qubits
    NUM_QUBITS_SUPPORTED = qubits
    dev = qml.device("qiskit.remote", wires=NUM_QUBITS_SUPPORTED, backend=backend)
except Exception as e:
    print(e)

In [107]:
from sklearn.model_selection import train_test_split
import pennylane as qml
import numpy as np
import pennylane as qml
from pennylane import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [113]:
# Load and prepare data
file = pd.read_csv(r'C:\Users\muthu\Downloads\sample.csv')
X = file.iloc[:, :8].values
Y = file.iloc[:, 8].values
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=0)


# Quantum kernel configuration
n_qubits = qubits  # Matches your 16-dimensional data
reps = 2       # Same as Qiskit's ZZFeatureMap reps

In [115]:
# Custom quantum feature map (equivalent to ZZFeatureMap)
def feature_map(x):
    for r in range(reps):
        # Encoding layer
        for i in range(n_qubits):
            qml.Hadamard(wires=i)
            qml.RZ(x[i], wires=i)

        # Entanglement layer (linear)
        for i in range(n_qubits-1):
            qml.CNOT(wires=[i, i+1])
            qml.RZ((x[i] * x[i+1]), wires=i+1)
            qml.CNOT(wires=[i, i+1])

# Quantum kernel circuit
@qml.qnode(dev)
def kernel_circuit(x1, x2):
    feature_map(x1)
    qml.adjoint(feature_map)(x2)
    return qml.probs(wires=range(n_qubits))

# Kernel function
def kernel(x1, x2):
    return kernel_circuit(x1, x2)[0]  # Probability of all-zero state

In [117]:
# Compute kernel matrices

print("Computing training kernel matrix...")
K_train = qml.kernels.square_kernel_matrix(X_train, kernel, assume_normalized_kernel=True)

print("Computing test kernel matrix...")
K_test = qml.kernels.kernel_matrix(X_test, X_train, kernel)

# Train classical SVM with quantum kernel
model = SVC(kernel='precomputed')
model.fit(K_train, Y_train)
score = model.score(K_test, Y_test)

print(f"\nClassification test score: {score:.4f}")

Computing training kernel matrix...
Computing test kernel matrix...


IBMRuntimeError: 'Error closing session: "(\'Connection aborted.\', RemoteDisconnected(\'Remote end closed connection without response\'))"'

In [ ]:
# Load the test dataset (PCA-transformed data)
File_test = pd.read_csv(r'C:\Users\muthu\Downloads\sample.csv')
X_new = File_test.iloc[:, :15].values
X_new.shape

In [ ]:
# Make predictions
predictions = model.predict(X_new)
print("Predictions on new data:", predictions)

In [ ]:
Y_train